In [1]:
!pip install transformers
!pip install evaluate
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 8.7 MB/s eta 0:00:00


In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import Dataset, load_dataset
import numpy as np
import pandas as pd
import evaluate
import wandb

In [13]:
dataset = load_dataset("imdb")
dataset = dataset['train'].train_test_split(test_size=0.2)

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [14]:
model_name = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
def tokenize(batch):
  return tokenizer(batch['text'], truncation=True, padding="max_length")

tokenized = dataset.map(tokenize, batched=True)

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [19]:
training_args = TrainingArguments(
    output_dir = "./results",
    #evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size = 16,
    num_train_epochs = 2,
    weight_decay = 0.01,
    logging_dir = './logs',
    logging_steps = 10,
)

In [20]:
metric = evaluate.load("accuracy")

def eval_metrics(eval):
  logits, labels = eval
  preds = np.argmax(logits, axis=1)
  return metric.compute(predictions = preds, references = labels)

In [21]:
trainer = Trainer(
    model = model,
    args = training_args,
    tokenizer = tokenizer,
    train_dataset = tokenized['train'],
    eval_dataset = tokenized['test'],
    compute_metrics = eval_metrics
)

/tmp/ipython-input-506597754.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
wandb.init()
trainer.train()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: akhyar (akhyar-university-of-engineering-and-technology-lahore) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss
10,0.692200
20,0.683000
30,0.639900
40,0.505600
50,0.409600
60,0.327700
70,0.333900
80,0.221800
90,0.396300
100,0.320600


TrainOutput(global_step=2500, training_loss=0.20676461759805678, metrics={'train_runtime': 1959.1461, 'train_samples_per_second': 20.417, 'train_steps_per_second': 1.276, 'total_flos': 5298695946240000.0, 'train_loss': 0.20676461759805678, 'epoch': 2.0})

In [23]:
result = trainer.evaluate()

In [24]:
print(result)

{'eval_loss': 0.25018197298049927, 'eval_accuracy': 0.9272, 'eval_runtime': 79.2854, 'eval_samples_per_second': 63.063, 'eval_steps_per_second': 3.948, 'epoch': 2.0}


In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
The token `Colab_Notebooks_Write` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to 

In [ ]:
trainer.push_to_hub("akhyar919/model-name")
tokenizer.push_to_hub("akhyar919/model-name")

In [ ]:
text = "i am happy today !"
inputs = tokenizer(text, return_tensors="pt")

# Move inputs to the same device as the model
inputs = {name: tensor.to(model.device) for name, tensor in inputs.items()}

outputs = model(**inputs)
print(outputs.logits.argmax(dim=1))

tensor([3], device='cuda:0')
